In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
from io import BytesIO
import base64
import numpy as np
from PIL import Image, ImageOps

In [3]:
url = "https://doodle-jump.io/game/doodle-jump/"
options = Options()
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=644,474")
#options.add_argument("--window-size=200,300")
#driver = webdriver.Chrome("chromedriver", options=options)

play = np.load("play_s.npy")
menu = np.load("menu_s.npy")

In [4]:
def get_screenshot(driver):
    image_b64= driver.get_screenshot_as_base64()
    screen = Image.open(BytesIO(base64.b64decode(image_b64))).crop((200,0,400,300))
    # elem_canvas = driver.find_element(By.XPATH, "/html/body/canvas")
    # screen = Image.open(BytesIO(elem_canvas.screenshot_as_png))
    return screen

In [5]:
from pynput import keyboard
from pynput.keyboard import Key
from threading import Thread 
from time import sleep
import time

def on_press(key):
    global current_key
    current_key = key

def on_release(key):
    pass


In [6]:
def get_screenshot_grayscale(driver):
    return ImageOps.grayscale(get_screenshot(driver))

In [7]:
from selenium.webdriver.common.action_chains import ActionChains

def start_game(driver):
    def __contains_play_button():
        loading_frame = np.array(get_screenshot_grayscale(driver)
            .crop((40, 72, 110, 97)))
        return np.array_equal(loading_frame, play)

    while True:
        driver.get(url)
        i = 0
        while i < 50 and not __contains_play_button():
            print('wait')
            sleep(0.1)
        i += 1
        if i < 50:
            elem_canvas = driver.find_element(By.XPATH, "/html/body/canvas")
            action_chains = ActionChains(driver)
            action_chains.move_to_element(elem_canvas).move_by_offset(-10, -75).click().perform()
        return


In [8]:
def game_over(driver):
    frame = get_screenshot_grayscale(driver)
    new_menu_frame = np.array(frame.crop((120, 210, 200, 250)))
    return np.array_equal(new_menu_frame, menu)

In [9]:
from threading import Thread
import os

def store_episode(states, actions):
    max = 0
    for f in os.listdir('./human_db'):
        e_n = int(f.split('.')[-2])
        if e_n > max:
            max = e_n
    
    # np.save(f'./human_db/states.{max + 1}', states)
    # np.save(f'./human_db/actions.{max + 1}', actions)
    np.savez_compressed(f'./human_db/episode.{max+1}', states=states, actions=actions)


In [41]:
driver = webdriver.Chrome("chromedriver", options=options)

current_key = None
early_end = None

listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()

steps = 215
states = np.zeros((steps, 2, 300, 200), dtype=np.single)
actions = np.ones(steps, dtype=np.int_) * -1

def capture_step(driver, i):
    global current_key
    global early_end

    for j in range(2):
        states[i,j] = np.array(get_screenshot_grayscale(driver), dtype=np.single)

    # 0: self.dJWB.move_left,
    # 1: self.dJWB.move_right,
    # 2: self.dJWB.stay_still,
    # 3: self.dJWB.shoot
    if current_key == Key.left:
        actions[i] = 0
    elif current_key == Key.right:
        actions[i] = 1
    elif current_key == None:
        actions[i] = 2
    elif current_key == Key.up:
        actions[i] = 3

    if current_key == Key.esc:
        early_end = i
        
    current_key = None

early_end = None

start_game(driver)
sleep(0.5)
#    while not game_over(driver):
start = time.time()

for i in range(steps):
    if early_end:
        break

    t = Thread(target=capture_step, args=(driver, i))
    t.start()
#    capture_step()
    sleep(0.7)
end = time.time()
print(f'duration: {end - start}')
driver.close()

states = states / 255

if early_end:
    states_t = np.empty((early_end, 2, 300, 200), dtype=np.single)
    states_t[:early_end] = states[:early_end]
    states = states_t

    actions_t = np.empty(early_end)
    actions_t[:early_end] = actions[:early_end]
    actions = actions_t

listener.stop()

store_episode(states, actions)

wait
wait
wait
wait
wait
wait
wait
duration: 91.23271131515503


In [11]:
et = np.load('./human_db/episode.6.npz')
print(et['actions'])
print(et['states'])

[2. 2. 1. 0. 0. 1. 2. 0. 0. 1. 1. 2. 0. 2. 0. 1. 2. 2. 2. 0. 2. 2. 2. 2.
 0. 1. 1. 1. 0. 1. 1. 2. 0. 0. 2. 1. 1. 2. 1. 2. 0. 0. 2. 1. 0. 2. 1. 1.
 2. 0. 2. 0. 2. 2. 0. 1. 2. 2. 3. 0. 2. 1. 2. 0. 1. 1. 0. 0. 1. 2. 0. 1.
 2. 0. 2. 0. 1. 2. 1. 0. 1. 2. 2. 0. 2. 1. 1. 2. 1. 2. 2. 2. 0. 0. 1. 2.
 1. 1. 2. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 0. 1.
 2. 2. 2. 0. 1. 2. 2. 2. 2. 2. 0. 1. 2. 1. 0. 1. 0. 2. 3. 3. 3. 2. 2. 2.
 2.]
[[[[0.7882353  0.8235294  0.827451   ... 0.83137256 0.8117647
    0.8235294 ]
   [0.79607844 0.8235294  0.827451   ... 0.827451   0.8117647
    0.8156863 ]
   [0.7882353  0.8235294  0.827451   ... 0.827451   0.80784315
    0.8117647 ]
   ...
   [0.8980392  0.94509804 0.9411765  ... 0.94509804 0.9137255
    0.9411765 ]
   [0.89411765 0.94509804 0.94509804 ... 0.9411765  0.9137255
    0.9411765 ]
   [0.8745098  0.89411765 0.8901961  ... 0.8901961  0.8784314
    0.8980392 ]]

  [[0.7882353  0.8235294  0.827451   ... 0.83137256 0.8117647
    0.8235294 ]
 

In [12]:
# driver = webdriver.Chrome("chromedriver", options=options)

# driver.get(url)

In [13]:
# np.save("play_s.npy", np.array(get_screenshot_grayscale(driver).crop((40, 72, 110, 97))))

In [14]:
# np.save("menu_s.npy", np.array(get_screenshot_grayscale(driver).crop((120, 210, 200, 250))))